# Calculate and submit days_from_study_to_pos/neg_covid_test
---
by Chris Meyer, PhD

Manager of Data and User Services at the Center for Translational Data Science at University of Chicago

November 2022

---
This Jupyter notebook demonstrates how to calculate the time interval in days between each of a patient's imaging studies and each of their positive/negative COVID-19 test results, which are the following properties on the imaging_study node in the [MIDRC data dictionary](https://github.com/uc-cdis/midrc_dictionary/blob/deae581f0fb8b9ae5add1458d7882e189ba97af6/gdcdictionary/schemas/imaging_study.yaml#L61):
* days_from_study_to_neg_covid_test
* days_from_study_to_pos_covid_test

# Install some Python packages:

In [1]:
# # ## May need to install some Python packages

# !pip install --upgrade pip
# !pip install --upgrade gen3


# Import Python Packages and scripts

In [2]:
# Import Python Packages and scripts
import pandas as pd
import sys, os, copy, datetime

import gen3
from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query


In [3]:
# Set your working directory and change to it
temp_dir = "/Users/christopher/Documents/Notes/MIDRC/temporal"
os.chdir(temp_dir)


In [4]:
# download and import some custom Python scripts from https://github.com/cgmeyer/gen3sdk-python
os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py")
from expansion import Gen3Expansion


In [5]:
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://staging.midrc.org/identity
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
api = 'https://staging.midrc.org'
cred = '/Users/christopher/Downloads/midrc-staging-credentials.json'
auth = Gen3Auth(api, refresh_file=cred) # authentication class
sub = Gen3Submission(api, auth) # submission class
query = Gen3Query(auth) # query class
exp = Gen3Expansion(api,auth,sub) # class with some custom scripts
exp.get_project_ids()


Getting all project_ids you have access to in the data commons.
['Open-A1', 'Open-A1_PETAL_REDCORAL', 'Open-R1', 'TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD']


/Users/christopher/Documents/Notes/MIDRC/temporal/expansion.py:213: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(res["data"]["project"])


['Open-A1',
 'Open-A1_PETAL_REDCORAL',
 'Open-R1',
 'TCIA-COVID-19-AR',
 'TCIA-COVID-19-NY-SBU',
 'TCIA-COVID-19_CT_Images',
 'TCIA-RICORD']

## Export metadata using submission API
---
Here we'll utilize the MIDRC submission API (the "sheepdog" Gen3 service) to export all the imaging study and measurement (COVID-19 tests) data using the ["Gen3Expansion.get_node_tsvs" function](https://github.com/cgmeyer/gen3sdk-python/blob/2aecc6575b22f9cca279b650914971dd6723a2ce/expansion/expansion.py#L219), which is a wrapper to export and merge all the records in a node across each project in the data commons using the [Gen3SDK](https://github.com/uc-cdis/gen3sdk-python/) function [Gen3Submission.export_node()](https://github.com/uc-cdis/gen3sdk-python/blob/5d7b5270ff11cf7037f211cf01e410d8e73d6b84/gen3/submission.py#L361).

In [ ]:
# We only need to get data from the Open-A1/R1 projects; can add other projects here if applicable in the future.
projects = ["Open-A1","Open-R1"]


In [ ]:
# Export all the records in the imaging_study node for specified projects. 
# Make sure to use the "overwrite=True" option so you're working with the newest version of the data.
# This function will return a "master" dataframe with merged data from all projects; also saves TSVs in "outdir" directory
st = exp.get_node_tsvs(projects=projects, node="imaging_study", overwrite=True, outdir="staging_tsvs")


In [ ]:
# Now export all the data in the measurement node, which is used to store the COVID test data
meas = exp.get_node_tsvs(projects=projects, node='measurement', overwrite=True, outdir="staging_tsvs")


In [ ]:
cases = list(set(meas['cases.submitter_id']) & set(st['cases.submitter_id'])) # intersection of case IDs in imaging study and measurement nodes
print("Imaging study and measurement data available for {} case IDs.".format(len(cases)))


## Calculate the days from each imaging study to each COVID-19 test.
---
Now that we have the temporal data for imaging studies and COVID-19 tests in a single DataFrame for all cases in MIDRC for which this data is provided, we can calculate the number of days between each COVID-19 test and each imaging study, which we'll call `days_from_study_to_test`.

* Note: In MIDRC, a negative "days to XYZ" indicates that the event "XYZ" took place that many days prior to the index event, while a positive "days to" indicates the number of days since the index event. For example, a "days_to_study" of "-10" indicates that the imaging study was performed 10 days *before* the index event. A value of "365" indicates the imaging study took place one year *after* the index event. 
* Note: The `index_event` property is on the `case` node and is often the date of the first imaging exam or the date of the first COVID-19 test; all other temporal "days_to" or "days_from_" properties are in relation to that index event.

For `days_from_study_to_test`, we expect a positive value if the test was performed after the study, as the test date is forward in time in relation to the study, and a negative value if the test was performed prior to the study, as the test is backwards in time in relation to the study.

- So, if `test_days_from_index` is `1` and `days_to_study` is `4`, the `days_from_study_to_test` should be `-3`, which means the COVID test took place 3 days before the imaging study. 
- If the COVID test is on day 4 and the imaging study is on day 1, then the `days_from_study_to_test` is `3`, meaning the COVID test took place 3 days after the imaging study.


In [ ]:
"""Define a function for calculating the list of values given:
    - days_to_study (int): for an imaging study and 
    - days_to_tests (list of int): which is a list of "test_days_from_index" for each of the patient's COVID-19 tests.
    Returns the list "days_from_study_to_tests", which is the list of the difference between each COVID test's "test_days_from_index" and the study's "days_to_study"
"""
def get_days_from_study_to_tests(days_to_tests, days_to_study):
    days_from_study_to_tests = []
    if hasattr(days_to_tests, '__iter__'):
        for days_to_test in days_to_tests:
            days_from_study_to_tests.append(days_to_test - days_to_study)
        days_from_study_to_tests = list(set(days_from_study_to_tests))
        days_from_study_to_tests = sorted(days_from_study_to_tests, reverse=True)
        days_from_study_to_tests = ",".join(str(x) for x in days_from_study_to_tests).rstrip(',')
        return days_from_study_to_tests
    else:
        return days_to_tests


In [ ]:
# First we'll prepare the measurement data for the calculations.
# Filter out measurements with a null value for "test_days_from_index" and those without a test_name of "COVID-19".
m = copy.deepcopy(meas.loc[(~meas['test_days_from_index'].isna()) & (meas['test_name']=='COVID-19')])
display(m)


In [ ]:
# Group the measurement test_days_from_index based on the test_result_text
m = m.groupby(['cases.submitter_id','test_result_text']).test_days_from_index.apply(sorted).reset_index()
display(m)


In [ ]:
# Get a list of "test_days_from_index" for each case ID based on value of "test_result_text"
m = m.pivot(index='cases.submitter_id', columns='test_result_text', values='test_days_from_index').reset_index()
display(m)


In [ ]:
# Merge the imaging studies with the measurement temporal data
sdf = pd.merge(left = st, right = m, how = 'left', on = 'cases.submitter_id')
display(sdf)


In [ ]:
# Now calculate the derived properties by applying the lambda function "get_days_from_study_to_tests"
sdf['days_from_study_to_neg_covid_test'] = sdf.apply(lambda x: get_days_from_study_to_tests(x.Negative, x.days_to_study), axis=1)
sdf['days_from_study_to_pos_covid_test'] = sdf.apply(lambda x: get_days_from_study_to_tests(x.Positive, x.days_to_study), axis=1)
display(sdf[['days_from_study_to_pos_covid_test','days_from_study_to_neg_covid_test']])


In [ ]:
study_props = ['type','project_id','submitter_id','cases.submitter_id','days_from_study_to_neg_covid_test','days_from_study_to_pos_covid_test','datasets.submitter_id']
df = sdf[study_props]
display(df)


In [ ]:
# Filter out imaging studies that don't have a value for either temporal property
df = df.loc[(~df['days_from_study_to_neg_covid_test'].isna()) | (~df['days_from_study_to_pos_covid_test'].isna())].reset_index().drop(columns="index")
display(df)


In [ ]:
now = datetime.datetime.now()
today = "{}-{}-{}".format(now.year, now.month, now.day)
filename="temporal_imaging_study_{}_{}.tsv".format(len(df),today)
df.to_csv(filename, sep='\t', index=False)
print("Master temporal data for {} imaging studies saved to file: {}".format(len(df),filename))


In [ ]:
# Submit the derived data to staging
data = {}
for pid in projects:
    data[pid] = exp.submit_df(df=df.loc[df["project_id"]==pid],project_id=pid, chunk_size=1000)
